# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./data/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [2]:
# Drop unnecessary columns
new_df = df.drop(columns='timestamp')
new_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [3]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 
reader = Reader()
data = Dataset.load_from_df(new_df,reader)
data


Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [4]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [5]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [6]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run

params = {'n_factors':[20,50,100],
          'reg_all':[0.02,0.05,0.1]}
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)

In [10]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
# param_grid = {'n_factors':[20, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
#               'reg_all': [0.4, 0.6]}
# gs_model = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
# gs_model.fit(data)

In [8]:
# print out optimal parameters for SVD after GridSearch
print(g_s_svd.best_score)
print(g_s_svd.best_params)

{'rmse': 0.8699026896210308, 'mae': 0.6684119814803445}
{'rmse': {'n_factors': 50, 'reg_all': 0.05}, 'mae': {'n_factors': 50, 'reg_all': 0.05}}


In [9]:
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name':'pearson','user_based':True})
cv_knn_basic = cross_validate(knn_basic,data,n_jobs=-1)


In [13]:
# print out the average RMSE score for the test set
for i in cv_knn_basic.items():
    print(i)
    print('-'*40)
    print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.97910749, 0.96129526, 0.97026375, 0.97672827, 0.9749067 ]))
----------------------------------------
0.9724602962224942
('test_mae', array([0.75516057, 0.74151092, 0.74787967, 0.75977489, 0.75206068]))
----------------------------------------
0.9724602962224942
('fit_time', (1.3473622798919678, 1.4381542205810547, 1.4940059185028076, 1.5079660415649414, 1.4950048923492432))
----------------------------------------
0.9724602962224942
('test_time', (2.4664082527160645, 2.4773788452148438, 2.394599199295044, 2.446460247039795, 2.3427364826202393))
----------------------------------------
0.9724602962224942


In [10]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options = {'name':'pearson','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data,n_jobs=-1)

In [15]:
# print out the average score for the test set
for i in cv_knn_baseline.items():
    print(i)
    print('-'*40)
    print(np.mean(cv_knn_baseline['test_rmse']))

('test_rmse', array([0.86579719, 0.87429732, 0.88558868, 0.87935348, 0.8810276 ]))
----------------------------------------
0.8772128540473583
('test_mae', array([0.66171768, 0.66913047, 0.67449196, 0.67315406, 0.67226646]))
----------------------------------------
0.8772128540473583
('fit_time', (1.4202303886413574, 1.4630544185638428, 1.5228936672210693, 1.4650847911834717, 1.4211981296539307))
----------------------------------------
0.8772128540473583
('test_time', (3.091708183288574, 3.089742660522461, 3.122680902481079, 3.061814308166504, 3.0528676509857178))
----------------------------------------
0.8772128540473583


Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [11]:
df_movies = pd.read_csv('./data/movies.csv')

In [12]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [13]:
svd = SVD(n_factors=50,reg_all=0.05)
svd.fit(dataset)

In [14]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.163076859517494, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [15]:
def movie_rater(movie_df,num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list

In [17]:
# try out the new function here!
user_rating = movie_rater(df_movies,4,'Comedy')
user_rating

      movieId                          title  genres
5251     8611  Farmer's Daughter, The (1947)  Comedy
      movieId           title  genres
1139     1490  B*A*P*S (1997)  Comedy
      movieId                                  title                genres
3346     4533  Return of the Living Dead, The (1985)  Comedy|Horror|Sci-Fi
      movieId                title                               genres
8397   110130  Nut Job, The (2014)  Adventure|Animation|Children|Comedy


[{'userId': 1000, 'movieId': 8611, 'rating': '2'},
 {'userId': 1000, 'movieId': 1490, 'rating': '3'},
 {'userId': 1000, 'movieId': 4533, 'rating': '4'},
 {'userId': 1000, 'movieId': 110130, 'rating': '5'}]

If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [18]:
user_rating

[{'userId': 1000, 'movieId': 8611, 'rating': '2'},
 {'userId': 1000, 'movieId': 1490, 'rating': '3'},
 {'userId': 1000, 'movieId': 4533, 'rating': '4'},
 {'userId': 1000, 'movieId': 110130, 'rating': '5'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [21]:
## add the new ratings to the original ratings DataFrame
user_rating =pd.DataFrame(user_rating)
new_ratings_df =pd.concat([new_df,user_rating],axis=0)
new_data = Dataset.load_from_df(new_ratings_df,reader)

In [24]:
# train a model using the new combined DataFrame
svd_ = SVD(n_factors=50,reg_all=0.05)
svd_.fit(new_data.build_full_trainset())

In [25]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
list_of_movies = []
for m_id in new_df['movieId'].unique():
    list_of_movies.append( (m_id,svd_.predict(1000,m_id)[3]))

In [28]:
list_of_movies

[(1, 3.926099110155218),
 (3, 3.248722939322864),
 (6, 3.867530778108566),
 (47, 4.046965335763656),
 (50, 4.153200543419315),
 (70, 3.528768442615093),
 (101, 3.934853738716591),
 (110, 4.188040660890699),
 (151, 3.5469421285636438),
 (157, 3.2757332583710115),
 (163, 3.5720443966463327),
 (216, 3.399292275497467),
 (223, 3.9357091761641176),
 (231, 3.077618366414768),
 (235, 3.6518740994616037),
 (260, 4.2761566580030195),
 (296, 4.300668530476791),
 (316, 3.3380197133063394),
 (333, 3.7034656478726595),
 (349, 3.499763162529489),
 (356, 4.17069298839117),
 (362, 3.602123267067119),
 (367, 3.2923313164037986),
 (423, 3.1995490643837896),
 (441, 4.077015697165933),
 (457, 4.11432154626264),
 (480, 3.766334395636466),
 (500, 3.4579225031667598),
 (527, 4.137775081196574),
 (543, 3.4512079252518957),
 (552, 3.33557272934902),
 (553, 3.7958893793463906),
 (590, 3.8056523236304636),
 (592, 3.5844742275517807),
 (593, 4.161452012360781),
 (596, 3.6401985396249628),
 (608, 4.082563928025061

In [29]:
# order the predictions from highest to lowest rated
ranked_movies = sorted(list_of_movies, key=lambda x:x[1], reverse=True)
ranked_movies

[(318, 4.345610444011098),
 (904, 4.3435767290246785),
 (750, 4.3323034626167525),
 (7153, 4.332024460750336),
 (3468, 4.327187866925631),
 (2329, 4.3254078604349875),
 (3275, 4.320864768982025),
 (1197, 4.319857448494982),
 (1204, 4.317102848357845),
 (58559, 4.311743584751177),
 (296, 4.300668530476791),
 (898, 4.298494649086702),
 (908, 4.293012449206433),
 (1233, 4.292552329489183),
 (1208, 4.292074543637643),
 (1217, 4.284187629550606),
 (260, 4.2761566580030195),
 (1252, 4.274980225247547),
 (1237, 4.271883742565019),
 (1104, 4.268522929690837),
 (1178, 4.264282998554862),
 (858, 4.263581497009153),
 (912, 4.260407588870004),
 (1136, 4.258434381924606),
 (2324, 4.254432066307795),
 (2959, 4.253454123235938),
 (541, 4.253281417115733),
 (1225, 4.25037594014201),
 (48516, 4.247216107769338),
 (1222, 4.242671334912404),
 (1213, 4.2414434767134965),
 (1250, 4.240573990117968),
 (6711, 4.237174842288878),
 (3266, 4.236487007817495),
 (38061, 4.228525021803752),
 (1221, 4.2276473888150

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [27]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation # ', idx+1, ': ', title, '\n')
            n-= 1
            if n == 0:
                break
            
recommended_movies(ranked_movies,df_movies,5)

Recommendation #  1 :  277    Shawshank Redemption, The (1994)
Name: title, dtype: object 

Recommendation #  2 :  686    Rear Window (1954)
Name: title, dtype: object 

Recommendation #  3 :  602    Dr. Strangelove or: How I Learned to Stop Worr...
Name: title, dtype: object 

Recommendation #  4 :  4800    Lord of the Rings: The Return of the King, The...
Name: title, dtype: object 

Recommendation #  5 :  2593    Hustler, The (1961)
Name: title, dtype: object 



## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.